# Dark Souls II Reviews (2025)

In [1]:
from get_data import *

In [2]:
reviews = get_data()

## Topic Modeling:
- Exploring certain aspects on why people like the game
    - Also get critiques of the game in positive reviews (if any but there sure is considering DS2's reputation in the community)

- Exploring why people don't like the game:
    - Also get positive aspects within this subset of the reviews

Splitting the reviews by how many do and don't recommend buying the game:

In [3]:
pos_reviews = reviews[reviews['voted_up'] == True]
neg_reviews = reviews[reviews['voted_up'] == False]

In [4]:
count_vector = CountVectorizer()

tf = count_vector.fit_transform(reviews.review)
tf_feat_names = count_vector.get_feature_names_out()

pos_tf = count_vector.fit_transform(pos_reviews.review) 
pos_tf_feat_names = count_vector.get_feature_names_out()

neg_tf = count_vector.fit_transform(neg_reviews.review)
neg_tf_feat_names = count_vector.get_feature_names_out()

In [5]:
pos_reviews.shape, neg_reviews.shape

((55777, 10), (11222, 10))

### LSA:

In [6]:
from sklearn.decomposition import TruncatedSVD

In [7]:
count_vector = CountVectorizer()
n_topics = 4

In [8]:
lsa = TruncatedSVD(n_components=n_topics, random_state=42)
lsa_matrix = lsa.fit_transform(tf)

def display_topics(model, feature_names, num_top_words=5):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(lsa, tf_feat_names)

Topic 0:
death like get enemy play
Topic 1:
keep iron hate die go
Topic 2:
die like get play dy
Topic 3:
trash fuck like play get


In [9]:
pos_tf = count_vector.fit_transform(pos_reviews.review)
pos_lsa_matrix = lsa.fit_transform(pos_tf)
pos_tf_feat_names = count_vector.get_feature_names_out()
print("\nPositive Reviews Topics:")
display_topics(lsa, pos_tf_feat_names)


Positive Reviews Topics:
Topic 0:
death like get die play
Topic 1:
die dy get like play
Topic 2:
keep iron hate go never
Topic 3:
rolin die death dy meatslap


In [10]:
neg_tf = count_vector.fit_transform(neg_reviews.review)
neg_lsa_matrix = lsa.fit_transform(neg_tf)
neg_tf_feat_names = count_vector.get_feature_names_out()
print("\nNegative Reviews Topics:")
display_topics(lsa, neg_tf_feat_names)


Negative Reviews Topics:
Topic 0:
fuck trash like enemy get
Topic 1:
trash like good buy someone
Topic 2:
keep iron enemy like make
Topic 3:
enemy like play make get
